# 당근 데이터 전처리

## 0. data load & settings

In [43]:
import pandas as pd

df = pd.read_csv('../../data/raw/daangn.csv')
raw = df
df.head()

,id,title,detail,condition,uploaded_date,is_completed,price,location
0,fdffb5c6-480b-4891-b6d4-ff41b6fabd83,가격낮춤!(24년3월구매)스토케 익스플로리 엑스 디럭스 유모차,스토케 익스플로리 엑스 디럭스 유모차입니다. ☆보관장소가 협소해서 빠른거래위해 가격...,NaN,2024-11-24 15:15:35.209000,True,600000.0,천호동
1,98f6c307-e94f-4c78-9f6f-c1ab6045a2f3,가격내림제안주세요🩶스토케 비트 블랙멜란지 절충형 유모차 팔아요🩶+방풍커버,21년도에 아기 태어나면서 구매한 절충형 유모차 판매합니다 ! 백만원 넘게 주고 새...,NaN,2023-11-10 20:32:23.635000,True,200000.0,덕은동
2,d9036794-e558-46a6-a1e7-dbde3c900836,[가격내림] 스토케 스쿠트 유모차,스토케 스쿠트 유모차 구입일시: 2023년 7월 (재당근) 당근에서 구매한 스토케 ...,NaN,2024-05-29 21:07:26.426000,True,20000.0,등촌제2동
3,f02eaa6a-784b-4fa7-ab86-06edea7af02b,가격내림) 스토케 익스플로리 엑스 디럭스 유모차(거의새거),신세계 백화점 구매했는데 겨울에 구매해서 밖에 나가보지도 못하고 두고있다 둘째때 써...,NaN,2024-01-15 13:42:37.786000,True,650000.0,대저2동
4,18babe60-fa9e-4bad-a066-f8cd5c6eef56,(가격내림)스토케 크루시 디럭스 유모차,17년도 구매했습니다! 장바구니가 커서 사용하기 좋아요~ 유모차 전문업체 세탁 후 ...,NaN,2024-01-21 11:08:24.848000,True,45000.0,월성1동


## 1. condition
    1. title + detail에서 상태값 생성 
        1. 네이버 카테고리에 맞춰서 추가 ⇒ 사용감 있음, 거의 새 것, 새 상품
        2. 구할 수 없는 경우에 따른 카테고리 추가  ⇒ None
    2. "15,000원”과 같은 맞지 않는 정보가 있음 ⇒ 카테고리가 없으면 default값 None으로 처리됨

In [44]:
import re
import numpy as np

new_pattern = r"""
(?:새\s*상품|새\s*제품|신\s*상품|신상) |
(?:미\s*개봉|개봉\s*안함|개봉\s*하지\s*않음|개봉\s*X|언박싱\s*안함) |
(?:미\s*사용|사용\s*안함|사용\s*하지\s*않음) |
(?:박스\s*(?:채|그대로|보관)) |
(?:택\s*(?:달림|부착))
"""

almost_new_pattern = r"""
(?:거의\s*새\s*(?:거|것))|
(?:최상(?:급)?|상태\s*최상(?:급)?|최상태)|
(?:S\s*급|A\s*급|상태\s*(?:S|A)\s*급)|
(?:깨끗|깔끔|하자\s*없음)|
(?:실\s*사용)|
(?:사용\s*감\s*(?:거의\s*)?없음)|
(?:(?:한\s*두|1|두)\s*번\s*사용|몇\s*번\s*사용|1\s*회\s*사용|사용\s*횟수\s*적음)|
(?:보관\s*만|전시\s*만|보관만함|전시만함)
"""

old_pattern = r"""
(?:흠집|자국|찍힘|찢어짐|찢김|깨짐|파손|부러짐)|
(?:중고|오래\s*됨|오래됨)|
(?:사용\s*감|생활\s*감|사용\s*흔적|사용흔함|사용흔적\s*많음)|
(?:기스|스크래치|찌그러짐|눌림|헐거움|유격|느슨)|
(?:변색|색\s*바램|바램|바랜|색바램)|
(?:오염|얼룩|때\s*(?:탐|낌|탔다))|
(?:해짐|닳음|낡음|낡은|낡았)|
(?:소음|삐걱|유격)|
(?:리폼|수리|보수|수선|교체|부품\s*없음|부품\s*분실)|
(?:보증\s*만료|보증\s*없음|무상\s*수리\s*기간\s*끝)|
(?:재\s*당근|연식)|
(?:20(?:1[5-9]|2[0-3])년(?:식|도)?)|
(?:\d{2}\s*년\s*식)
"""

new_mask = (
    df['title'].str.contains(new_pattern, case=False, flags=re.VERBOSE) |
    df['detail'].str.contains(new_pattern, case=False, flags=re.VERBOSE)
)
almost_new_mask = (
    df['title'].str.contains(almost_new_pattern, case=False, flags=re.VERBOSE) |
    df['detail'].str.contains(almost_new_pattern, case=False, flags=re.VERBOSE)
)
old_mask = (
    df['title'].str.contains(old_pattern, case=False, flags=re.VERBOSE) |
    df['detail'].str.contains(old_pattern, case=False, flags=re.VERBOSE)
)

# 우선순위 적용: 새상품 > 거의 새 것 > 사용감 있음 : '사용감 있음'이 가장 많음
df['condition'] = np.select(
    [new_mask, almost_new_mask, old_mask],
    ['새 상품', '거의 새 것', '사용감 있음'],
    default=None # 이상값이나 결측치 처리
)

df['condition'].isna().sum()

np.int64(1992)

## 2. location 
    1. 구로 묶어서 구별 가격 군집 분석
    2. 결측치(+유아동)의 경우 None으로 처리

In [45]:
import json

with open("city_key_donglist_value.json", "r", encoding="utf-8") as f:
    regions = json.load(f)

city_dongs = {}

for key, value in regions.items():
    city_dongs[key] = set(value)

"""
    읍/면/동 이름(location)을 받아 해당하는 city(ex:서울특별시)를 반환.
    없으면 'Nan' 반환.
"""
def find_city(location):
    for key, names in city_dongs.items():
        if location in names:
            return key

    return None

df['location'] = df['location'].apply(find_city)
df['location'].isna().sum() # 법정동 이슈(검색은 행정동 기준인데, 상세는 법정동도 나와서 결측치 발생)

np.int64(1326)

## 3. title/detail 
    1. 이모지가 존재하므로 제거하고 사용
    2. detail에  html 태그만 들어간 경우 제거(소수)

In [46]:
import re

# html 제거용 정규식
html_pattern = re.compile(r'<.*?>')

# 이모지 제거용 정규식
emoji_pattern = re.compile("[" 
    u"\U0001F600-\U0001F64F"  # emoticons
    u"\U0001F300-\U0001F5FF"  # symbols & pictographs
    u"\U0001F680-\U0001F6FF"  # transport & map
    u"\U0001F1E0-\U0001F1FF"  # flags
    u"\u2700-\u27BF"          # dingbats
    u"\U0001F900-\U0001F9FF"  # supplemental symbols
    u"\U0001FA70-\U0001FAFF"  # extended-A
    u"\u2600-\u26FF"          # ★ 추가: Misc Symbols (☀, ☺ 등)
    u"\u2B50"                 # ★ 추가: ⭐
    u"\u2B55"                 # ★ 추가: ⭕
    u"\u203C"                 # ★ 추가: ‼
    u"\uFE0E-\uFE0F"          # ★ 추가: variation selectors (️ 잔여 제거)
    "]+", flags=re.UNICODE)

def clean_text(text):
    no_html = html_pattern.sub('', text)      # HTML 제거
    no_emoji = emoji_pattern.sub('', no_html) # 이모지 제거
    return no_emoji.strip()

df["title"] = df["title"].apply(clean_text)
df["detail"] = df["detail"].apply(clean_text)

In [47]:
df.info()
df.to_csv('daangn_clean_condition.csv', index=False, encoding='utf-8-sig')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5836 entries, 0 to 5835
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             5836 non-null   object 
 1   title          5836 non-null   object 
 2   detail         5836 non-null   object 
 3   condition      3844 non-null   object 
 4   uploaded_date  5836 non-null   object 
 5   is_completed   5836 non-null   bool   
 6   price          5836 non-null   float64
 7   location       4510 non-null   object 
dtypes: bool(1), float64(1), object(6)
memory usage: 325.0+ KB
